In [ ]:
# Usual imports
import secml
import numpy as np
from tqdm import tqdm
from scipy.special import softmax
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from joblib import Parallel, delayed
import pickle
import os
import pandas as pd
import csv
from torchvision import datasets, transforms
import shutil
import json
from PIL import Image
import random
import seaborn as sns

# SecML
from secml.ml.features.normalization import CNormalizerMinMax
from secml.ml.peval.metrics import CMetricAccuracy
from secml.array import CArray
from secml.data import CDataset
from secml.ml.classifiers import CClassifierPyTorch

# RobustBench
import robustbench
from robustbench.utils import load_model
from secml.utils import fm
from secml import settings

# Albi utils
from utils_attacks import *
from utils_CP import *
from utils_train_tinyimagenet import *

# Torchvision
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset
import torchvision.datasets as datasets

2025-04-16 22:16:20.266355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-16 22:16:20.444292: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-16 22:16:20.449861: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/torch/lib:/usr/local/lib/python3.10/dist-pa

## Dataset

#### Procedure to download, split and preprocess ImageNet (done once, then pickle upload)

In [3]:

# Path to the root directory containing both the validation images and devkit
#dataset_path = "/home/acarlevaro/Sources/albi/Adversarial_CP_V3/InyImageNet"

# Define the transforms (resize, tensor conversion, and normalization)
#transform = transforms.Compose([
#    transforms.Resize((224, 224)),  # Resize to 224x224 (ImageNet standard)
#    transforms.ToTensor(),
#    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
#])

#transform = transforms.Compose([
#    transforms.Resize(256),
#    transforms.CenterCrop(224),
#    transforms.ToTensor(),
#])

# Load ImageNet Validation Dataset
#val_dataset = datasets.ImageNet(
#    root=dataset_path,
#    split='val',  # You can also use 'train' for the training dataset
#    transform=transform
#)

# Example of iterating over the dataset
#for img, label in val_dataset[:50,:]:
#    print(img.size(), label)
    
dataset_path = "/home/acarlevaro/Sources/albi/Adversarial_CP_V3/InyImageNet/ILSVRC2012_Albi"

# Define the transforms (resize, tensor conversion, and normalization)
#transform = transforms.Compose([
#    transforms.Resize((224, 224)),  # Resize to 224x224 (ImageNet standard)
#    transforms.ToTensor(),
#    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
#])

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Select a random subset of 10,000 samples
num_samples = 10000
subset_indices = random.sample(range(len(full_dataset)), num_samples)
subset_dataset = Subset(full_dataset, subset_indices)

learning_dataset = subset_dataset

#X_train = np.array([train_dataset[i][0].numpy() for i in range(len(train_dataset))])
#y_train = np.array([train_dataset[i][1] for i in range(len(train_dataset))])

X_learning = np.array([learning_dataset[i][0].numpy() for i in range(len(learning_dataset))])
y_learning = np.array([learning_dataset[i][1] for i in range(len(learning_dataset))])

#print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")  
print(f"X_learning shape: {X_learning.shape}, y_learning shape: {y_learning.shape}")  

# Convert NumPy arrays to CArray
X_learning_carray = CArray(X_learning.reshape(X_learning.shape[0], -1))  # Flatten images
y_learning_carray = CArray(y_learning)

# Create the CDataset
lr = CDataset(X_learning_carray, y_learning_carray)

# Print dataset summary
print("CDataset:", lr)

# Shuffle before splitting
random_state = 999
rng = np.random.default_rng(seed=random_state)
shuffled_indices = rng.permutation(lr.X.shape[0]).tolist()
lr = lr[shuffled_indices, :]


X_learning shape: (10000, 3, 224, 224), y_learning shape: (10000,)
CDataset: CDataset{'X': CArray(10000, 150528)(dense: [[0.992157 0.992157 0.992157 ... 0.992157 0.992157 0.996078] [0.745098 0.737255 0.733333 ... 0.607843 0.607843 0.603922] [0.337255 0.32549  0.294118 ... 0.066667 0.047059 0.027451] ... [0.270588 0.301961 0.337255 ... 0.286275 0.286275 0.298039] [0.254902 0.254902 0.235294 ... 0.14902  0.145098 0.141176] [0.509804 0.513726 0.517647 ... 0.121569 0.152941 0.047059]]), 'Y': CArray(10000,)(dense: [885 520 195 ... 241 481 850]), 'header': None}


#### Pickle file saved

In [5]:
import pickle

# Save the CDataset object
with open('CD_ILSVRC2012_Albi.pkl', "wb") as f:
    pickle.dump(lr, f)
print("CD_ILSVRC2012_Albi.pkl saved to pickle file.")


CD_ILSVRC2012_Albi.pkl saved to pickle file.


## Load dataset (pickle, ImageNet)

In [2]:
import pickle

# Load the CDataset object
with open('CD_ILSVRC2012_Albi.pkl', "rb") as f:
    lr = pickle.load(f)

# Print dataset summary to verify
print("CDataset loaded successfully:", lr)

CDataset loaded successfully: CDataset{'X': CArray(10000, 150528)(dense: [[0.956863 0.952941 0.956863 ... 0.078431 0.105882 0.145098] [0.137255 0.152941 0.117647 ... 0.929412 0.823529 0.498039] [0.333333 0.305882 0.337255 ... 0.105882 0.082353 0.12549 ] ... [0.811765 0.811765 0.815686 ... 0.752941 0.756863 0.721569] [0.560784 0.552941 0.560784 ... 0.27451  0.282353 0.27451 ] [0.494118 0.505882 0.501961 ... 0.835294 0.819608 0.662745]]), 'Y': CArray(10000,)(dense: [208 102 898 ... 591 467 941]), 'header': None}


In [11]:
# Define split sizes
n_tr = 5000  # Training set
n_val = 50   # Validation set
n_ts = 1000  # Test set
n_cl = 2500  # Calibration set
n = n_tr + n_val + n_cl + n_ts

# Split dataset
tr = lr[:n_tr, :]
vl = lr[n_tr:n_tr + n_val, :]
cl = lr[n_tr + n_val:n_tr + n_val + n_cl, :]
ts = lr[n_tr + n_val + n_cl:n, :]

In [4]:
224*224*3

150528

## Load a Model

### Salman, ResNet-18, NeurIPS2020, RA: 25.32%

In [10]:
output_dir = fm.join(settings.SECML_MODELS_DIR, 'robustbench')
model_Salman = load_model(model_name='Salman2020Do_R18', dataset='imagenet', threat_model='Linf', model_dir=output_dir)

In [11]:
from secml.ml.features import CNormalizerMeanStd

# imagenet normalization
#normalizer = CNormalizerMeanStd(mean=(0.485, 0.456, 0.406),
#                                std=(0.229, 0.224, 0.225))

clf_Salman = CClassifierPyTorch(model = model_Salman,
                         input_shape=(3,224,224),
                         pretrained=True,
                         #preprocess=normalizer,
                         softmax_outputs = True)

from secml.ml.peval.metrics import CMetricAccuracy
metric = CMetricAccuracy()

# Compute predictions on a test set
y_pred = clf_Salman.predict(ts.X)

# Evaluate the accuracy of the classifier
acc = metric.performance_score(y_true=ts.Y, y_pred=y_pred)

print("Accuracy on test set: {:.2%}".format(acc))

Accuracy on test set: 50.00%


### Liu, Swin-L, arXivFeb2023, RA: 56.56%

In [12]:
output_dir = fm.join(settings.SECML_MODELS_DIR, 'robustbench')
model_Liu2023 = load_model(model_name='Liu2023Comprehensive_ConvNeXt-B', dataset='imagenet', threat_model='Linf', model_dir=output_dir)

In [13]:
from secml.ml.features import CNormalizerMeanStd

# imagenet normalization
#normalizer = CNormalizerMeanStd(mean=(0.485, 0.456, 0.406),
#                                std=(0.229, 0.224, 0.225))

clf_Liu = CClassifierPyTorch(model = model_Liu2023,
                         input_shape=(3,224,224),
                         pretrained=True,
                         #preprocess=normalizer,
                         softmax_outputs = True)

from secml.ml.peval.metrics import CMetricAccuracy
metric = CMetricAccuracy()

# Compute predictions on a test set
y_pred = clf_Liu.predict(ts.X)

# Evaluate the accuracy of the classifier
acc = metric.performance_score(y_true=ts.Y, y_pred=y_pred)

print("Accuracy on test set: {:.2%}".format(acc))

Accuracy on test set: 74.60%


### TIAN

In [14]:
output_dir = fm.join(settings.SECML_MODELS_DIR, 'robustbench')
model_Tian = load_model(model_name='Tian2022Deeper_DeiT-B', dataset='imagenet', threat_model='corruptions', model_dir=output_dir)

In [15]:
from secml.ml.features import CNormalizerMeanStd

# imagenet normalization
#normalizer = CNormalizerMeanStd(mean=(0.485, 0.456, 0.406),
#                                std=(0.229, 0.224, 0.225))

clf_Tian = CClassifierPyTorch(model = model_Tian,
                         input_shape=(3,224,224),
                         pretrained=True,
                         #preprocess=normalizer,
                         softmax_outputs = True)

from secml.ml.peval.metrics import CMetricAccuracy
metric = CMetricAccuracy()

# Compute predictions on a test set
y_pred = clf_Liu.predict(ts.X)

# Evaluate the accuracy of the classifier
acc = metric.performance_score(y_true=ts.Y, y_pred=y_pred)

print("Accuracy on test set: {:.2%}".format(acc))

Accuracy on test set: 74.60%


In [16]:
import shutil

def evaluate_attacks(model_name, norm_name, attack_configs, cl, ts, clf, alpha=0.1, base_output_dir="./Results"):
    """
    Evaluate different adversarial attacks on a given model and dataset.
    """
    n_cl = cl.X.shape[0]
    
    # Prepare output directory
    save_path = os.path.join(base_output_dir, model_name, norm_name)
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    os.makedirs(save_path, exist_ok=True)
    csv_file = os.path.join(save_path, "results_all.csv")
    
    # Run attacks
    cl_att_dict = attack_dataset(cl, clf, attack_configs, desc="Running attacks", n_jobs=1)
    ts_att_dict = attack_dataset(ts, clf, attack_configs, desc="Running attacks", n_jobs=1)
    
    # Process results
    results = []
    
    if isinstance(cl_att_dict, dict):
        attack_types = cl_att_dict.keys()
    else:
        attack_types = [get_single_attack_key(attack_configs[0])]
        cl_att_dict = {attack_types[0]: cl_att_dict}
        ts_att_dict = {attack_types[0]: ts_att_dict}
    
    for attack_type in attack_types:
        cl_att = cl_att_dict[attack_type]
        ts_att = ts_att_dict[attack_type]
        
        cl_att_scores = compute_score(cl, cl_att, clf)
        cl_scores = compute_score(cl, cl, clf)
        
        # Compute quantiles
        q_level = np.ceil((n_cl + 1) * (1 - alpha)) / n_cl
        qhat = np.quantile(cl_scores, q_level, method='higher')
        qhat_A = np.quantile(cl_att_scores, q_level, method='higher')
        
        # Compute conformal sets
        att_conformal_sets,_ = compute_CP(ts_att, qhat_A, clf)
        cs_conformal_sets,_ = compute_CP(ts_att, qhat, clf)
        
        # Compute coverage and variance
        att_coverage = compute_covergae(ts, att_conformal_sets)
        att_coverage_var = compute_covergae_std(ts, att_conformal_sets)
        cs_coverage = compute_covergae(ts, cs_conformal_sets)
        cs_coverage_var = compute_covergae_std(ts, cs_conformal_sets)
        
        # Compute mean and variance of set sizes
        att_size_mean = mean_conformal_sets(att_conformal_sets)
        att_size_var = std_conformal_sets(att_conformal_sets) / 10
        cs_size_mean = mean_conformal_sets(cs_conformal_sets)
        cs_size_var = std_conformal_sets(cs_conformal_sets) / 10
        
        results.append({
            "attack_type": "Vanilla",
            "coverage": f"{cs_coverage:.4f} ± {cs_coverage_var:.4f}",
            "size": f"{cs_size_mean:.4f} ± {cs_size_var:.4f}"
        })
        
        results.append({
            "attack_type": attack_type,
            "coverage": f"{att_coverage:.4f} ± {att_coverage_var:.4f}",
            "size": f"{att_size_mean:.4f} ± {att_size_var:.4f}"
        })
    
    # Save results to CSV
    file_exists = os.path.isfile(csv_file)
    with open(csv_file, mode="a", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["attack_type", "coverage", "size"])
        if not file_exists:
            writer.writeheader()
        writer.writerows(results)
    
    print(f"Results saved to {csv_file}")


### Liu2023

In [ ]:
model_name = "Liu"

#attack_configs = [ 
#    {"attack_type": "PGD", "epsilon": 0.5, "step_size": 0.5/10, "steps": 10, "distance": "l2", "lb": 0.0, "ub": 1.0},
#    {"attack_type": "FGM", "epsilon": 0.5, "distance": "l2", "lb": 0.0, "ub": 1.0},
#    {"attack_type": "DeepFool", "epsilon": 0.5, "distance": "l2"},
#    {"attack_type": "BasicIterative", "epsilon": 0.5, "distance": "l2"},
#    {"attack_type": "CW"}
#    #{"attack_type": "DDN", "epsilon": 0.5, "init_epsilon":0.01, "gamma":0.01, "steps":50, "lb":0.0, "ub":1.0},
#    #{"attack_type": "EAD", "epsilon": 0.5, "binary_search_steps":15, "initial_stepsize":0.01, "confidence":0.0, "initial_const":0.01, "regularization":0.1, "steps":10, "lb":0.0, "ub":1.0 }
#]

#evaluate_attacks(model_name, "L2", attack_configs, cl, ts, clf_Liu, alpha=0.1, base_output_dir="./Results")

eps_inf = 4/255

attack_configs_linf = [ 
    {"attack_type": "PGD", "epsilon": eps_inf, "step_size": eps_inf/10, "steps": 10, "distance": "linf", "lb": 0.0, "ub": 1.0},
    {"attack_type": "FGM", "epsilon": eps_inf, "distance": "linf", "lb": 0.0, "ub": 1.0},
    {"attack_type": "DeepFool", "epsilon": eps_inf, "distance": "linf"},
    {"attack_type": "BasicIterative", "epsilon": eps_inf, "distance": "linf"},

]

evaluate_attacks(model_name, "Linf", attack_configs_linf, cl, ts, clf_Liu, alpha=0.1, base_output_dir="./Results")

#eps_1 = 0.5*np.sqrt(224)

#attack_configs_l1 = [ 
#    {"attack_type": "PGD", "epsilon": eps_1, "step_size": eps_1/10, "steps": 10, "distance": "l1", "lb": 0.0, "ub": 1.0},
#    {"attack_type": "FGM", "epsilon": eps_1, "distance": "l1", "lb": 0.0, "ub": 1.0},
#    {"attack_type": "BasicIterative", "epsilon": eps_1, "distance": "l1"},

#]

#evaluate_attacks(model_name, "L1", attack_configs, cl, ts, clf_Liu, alpha=0.1, base_output_dir="./Results")

Running attacks:   0%|          | 0/500 [00:00<?, ?sample/s]

2025-04-16 22:19:43,171 - py.warnings - WARNING - /home/acarlevaro/.local/lib/python3.10/site-packages/torch/autograd/graph.py:823: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



### Salman

In [ ]:
model_name = "Salman"

attack_configs = [ 
    {"attack_type": "PGD", "epsilon": 0.5, "step_size": 0.5/10, "steps": 10, "distance": "l2", "lb": 0.0, "ub": 1.0},
    {"attack_type": "FGM", "epsilon": 0.5, "distance": "l2", "lb": 0.0, "ub": 1.0},
    {"attack_type": "DeepFool", "epsilon": 0.5, "distance": "l2"},
    {"attack_type": "BasicIterative", "epsilon": 0.5, "distance": "l2"},
    {"attack_type": "CW"}
    #{"attack_type": "DDN", "epsilon": 0.5, "init_epsilon":0.01, "gamma":0.01, "steps":50, "lb":0.0, "ub":1.0},
    #{"attack_type": "EAD", "epsilon": 0.5, "binary_search_steps":15, "initial_stepsize":0.01, "confidence":0.0, "initial_const":0.01, "regularization":0.1, "steps":10, "lb":0.0, "ub":1.0 }
]


evaluate_attacks(model_name, "L2", attack_configs, cl, ts, clf_Salman, alpha=0.1, base_output_dir="./Results")

eps_inf = 0.5/np.sqrt(224)

attack_configs_linf = [ 
    {"attack_type": "PGD", "epsilon": eps_inf, "step_size": eps_inf/10, "steps": 10, "distance": "linf", "lb": 0.0, "ub": 1.0},
    {"attack_type": "FGM", "epsilon": eps_inf, "distance": "linf", "lb": 0.0, "ub": 1.0},
    {"attack_type": "DeepFool", "epsilon": eps_inf, "distance": "linf"},
    {"attack_type": "BasicIterative", "epsilon": eps_inf, "distance": "linf"},

]

evaluate_attacks(model_name, "Linf", attack_configs_linf, cl, ts, clf_Salman, alpha=0.1, base_output_dir="./Results")

eps_1 = 0.5*np.sqrt(224)

attack_configs_l1 = [ 
    {"attack_type": "PGD", "epsilon": eps_1, "step_size": eps_1/10, "steps": 10, "distance": "l1", "lb": 0.0, "ub": 1.0},
    {"attack_type": "FGM", "epsilon": eps_1, "distance": "l1", "lb": 0.0, "ub": 1.0},
    {"attack_type": "BasicIterative", "epsilon": eps_1, "distance": "l1"},

]

evaluate_attacks(model_name, "L1", attack_configs, cl, ts, clf_Salman, alpha=0.1, base_output_dir="./Results")

### Tian

In [ ]:
model_name = "Tian"

attack_configs = [ 
    {"attack_type": "PGD", "epsilon": 0.5, "step_size": 0.5/10, "steps": 10, "distance": "l2", "lb": 0.0, "ub": 1.0},
    {"attack_type": "FGM", "epsilon": 0.5, "distance": "l2", "lb": 0.0, "ub": 1.0},
    {"attack_type": "DeepFool", "epsilon": 0.5, "distance": "l2", "steps": 10},
    {"attack_type": "BasicIterative", "epsilon": 0.5, "distance": "l2", "steps": 10},
    {"attack_type": "CW", "steps": 10}
    #{"attack_type": "DDN", "epsilon": 0.5, "init_epsilon":0.01, "gamma":0.01, "steps":50, "lb":0.0, "ub":1.0},
    #{"attack_type": "EAD", "epsilon": 0.5, "binary_search_steps":15, "initial_stepsize":0.01, "confidence":0.0, "initial_const":0.01, "regularization":0.1, "steps":10, "lb":0.0, "ub":1.0 }
]


evaluate_attacks(model_name, "L2", attack_configs, cl, ts, clf_Tian, alpha=0.1, base_output_dir="./Results")



Running attacks:   0%|          | 11/2500 [10:40<48:07:59, 69.62s/sample]